# Plotserver

The need to plot sensor and other data from microcontrollers occurs frequently. The plotserver plots data published to [MQTT](https://en.wikipedia.org/wiki/MQTT) in a browser window.

Let's look at the LinePlot application, installed in `~/plotserver/lib`. The `LinePlot` class runs in the plotserver service. It uses the [Bokeh Visualization Library](https://bokeh.org/) to render plots in a webbrowser. 

To plot a sinusoid, we first set up a proxy on the host (microcontroller, or, in this example, the Raspberry Pi). The first parameter in the *LinePlotProxy* constructor is the app name and must be a valid Python identified. The next parameter lists the name of the x-axis and of the traces that will be plotted. 

The optional *rollover* parameter controls how much data will be retained. Once that value is reached, the plot scrolls. The default is to retain all data.

In [3]:
import os, sys, math

sys.path.append(os.path.expandvars('$IOT_PROJECTS/plotserver/lib'))

from line_plot_proxy import LinePlotProxy

proxy = LinePlotProxy("sin", ['time', 'y'], rollover=50, title="Awesome sinusoid")
t = 0

serving app at https://server.local/plotserver/sin


Click on the link. An empty plot window shows up. The code below sends data to the plotserver. The plot window in the browser updates instantly.

In [4]:
for i in range(10):
    proxy.add_row([t, math.sin(0.1*t)])
    t += 1

Add more data ...

In [5]:
for i in range(80):
    proxy.add_row([t, math.sin(0.1*t)])
    t += 1

Note how the plot starts scrolling once *rollover* datapoints are displayed.

The code for *LinePlotProxy* is simple and can easily be ported to microcontrollers.

In [16]:
!cat $IOT_PROJECTS/plotserver/lib/line_plot_proxy.py

import paho.mqtt.client as mqtt
import json, urllib.request
import os, sys

sys.path.append(os.path.expandvars('$IOT_PROJECTS/plotserver/lib'))

from plot_proxy import PlotProxy
from config import *

class LinePlotProxy(PlotProxy):
    
    def __init__(self, app_name, column_names, *, rollover=None, update_interval=0.1, **figure_args):
        super().__init__(app_name, 
            "line_plot", "LinePlot", 
            column_names, rollover, update_interval, **figure_args)
        
    def add_row(self, row):
        topic = f"{TOPIC_ROOT}/call/{self._app_name}"
        payload = ("add_row", [row], {})
        self._mqttc.publish(topic, json.dumps(payload))


Reinitializing the proxy resets the plot and discards all data sent so far. Refresh the browser window to start anew.

The plotserver can handle up to ten applications (with different names) simultaneously. Once that limit is reached, requests for additional plots are ignored.

A list of currently open plots is available at

In [6]:
!echo https://$DNS_NAME.local/plotserver

https://server.local/plotserver


The `PlotProxy` initializer sends MQTT topic `start`, which turn creates a bokeh app in `$IOT_PROJECTS/plotserver/apps/PLOT_NAME`. E.g.

In [19]:
!cat $IOT_PROJECTS/plotserver/apps/sin.py

cat: /home/iot/iot49.org/docs/projects/plotserver/apps/sin.py: No such file or directory


Delete plots from that page or programmatically with

In [17]:
proxy.shutdown()

This also deletes the app in `$IOT_PROJECTS/plotserver/apps/PLOT_NAME`.